In [2]:
!pip install chromadb sentence-transformers

In [3]:
import chromadb
from sentence_transformers import SentenceTransformer

In [4]:
# Step 3: Load local embedding model (no remote download during add)
model = SentenceTransformer("all-MiniLM-L6-v2")
# Step 4: Prepare your documents and embeddings
documents = [
    "This document is about New York",
    "This document is about Delhi"
]

# Get embeddings from the model
embeddings = model.encode(documents).tolist()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Lenovo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# Step 5: Initialize Chroma client and collection
client = chromadb.Client()
collection = client.get_or_create_collection(name="my_collection")
# Step 6: Add documents with embeddings
collection.add(
    documents=documents,
    embeddings=embeddings,
    ids=['id1', 'id2']
)


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


In [6]:
# Step 7: Retrieve and print all documents
all_docs = collection.get()
print(all_docs)


Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


{'ids': ['id1', 'id2'], 'embeddings': None, 'documents': ['This document is about New York', 'This document is about Delhi'], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [None, None]}


In [7]:
# Fetch a specific document by ID
document = collection.get(ids=["id1"])
print(document)


{'ids': ['id1'], 'embeddings': None, 'documents': ['This document is about New York'], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [None]}


In [12]:
# Query the collection
results = collection.query(
    query_texts=['Query is about india'],
    n_results=2
)
print(results)


{'ids': [['id2', 'id1']], 'embeddings': None, 'documents': [['This document is about Delhi', 'This document is about New York']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None]], 'distances': [[0.7835128903388977, 1.5968999862670898]]}


In [13]:
# Delete all existing documents using their IDs
collection.delete(ids=all_docs['ids'])
print(collection.get())


Failed to send telemetry event CollectionDeleteEvent: capture() takes 1 positional argument but 3 were given


{'ids': [], 'embeddings': None, 'documents': [], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': []}


In [14]:
# Re-add documents with metadata
collection.add(
    documents=[
        "This document is about New York",
        "This document is about Delhi"
    ],
    ids=["id3", "id4"],
    metadatas=[
        {"url": "https://en.wikipedia.org/wiki/New_York_City"},
        {"url": "https://en.wikipedia.org/wiki/New_Delhi"}
    ],
    embeddings=model.encode([
        "This document is about New York",
        "This document is about Delhi"
    ]).tolist()  # Explicitly use local embedding
)


In [15]:
# Query with a culturally specific search
results = collection.query(
    query_texts=["Query is about Lotus temple"],
    n_results=2
)
print(results)


{'ids': [['id4', 'id3']], 'embeddings': None, 'documents': [['This document is about Delhi', 'This document is about New York']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'url': 'https://en.wikipedia.org/wiki/New_Delhi'}, {'url': 'https://en.wikipedia.org/wiki/New_York_City'}]], 'distances': [[1.1361644268035889, 1.6496894359588623]]}
